# Chainer cifar10 tutorial
自分の理解が深まるように少し改変  
https://github.com/pfnet/chainer/blob/master/examples/cifar/train_cifar.py

In [1]:
#!/usr/bin/env python
from __future__ import print_function
import argparse
import sys; sys.argv=['']; del sys

import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
from chainer import Variable
import copy # test iterator copy


In [2]:
# dataset = 'cifar10'
dataset = 'mnist'

batchsize = 128
epochsize = 300

# Load dataset

In [3]:
if dataset == 'cifar10':
    print('Using CIFAR10 dataset.')
    class_labels = 10
    train, test = chainer.datasets.get_cifar10()
elif dataset == 'cifar100':
    print('Using CIFAR100 dataset.')
    class_labels = 100
    train, test = chainer.datasets.get_cifar100()
elif dataset == 'mnist':
    print('Using mnist dataset.')
    class_labels = 10
    train, test = chainer.datasets.get_mnist()
    train = [(it[0].reshape(1, 28, 28),it[1]) for it in train]
    test = [(it[0].reshape(1, 28, 28),it[1]) for it in test]

Using mnist dataset.


In [4]:
print(len(train))
print(len(test))

60000
10000


In [5]:
train[0][0].shape

(1, 28, 28)

In [6]:
type(train[0][0])

numpy.ndarray

In [7]:
# train = [(it[0].flatten(),it[1]) for it in train]
# test = [(it[0].flatten(),it[1]) for it in test]

In [8]:
# Load the cifar dataset
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)
testsize = len(test)
testsize

10000

# Network

In [18]:
class VGG(chainer.Chain):
    def __init__(self, class_labels):
        initializer = chainer.initializers.HeNormal()
        super(VGG, self).__init__(
             # the size of the inputs to each layer will be inferred
            conv1=L.Convolution2D(None, 32, 3, stride=1, pad=0),
            conv2=L.Convolution2D(32, 64, 3, stride=1, pad=0),
            fc3=L.Linear(64*6*6, 1000, initialW=initializer),
            fc4=L.Linear(1000, class_labels, initialW=initializer),
        )

    def __call__(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), 2, stride=2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2, stride=2)        
        h = self.fc3(h)
        y = self.fc4(h)        
        return y

In [19]:
class Classifier(chainer.Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__(predictor=predictor)
        
    def clear(self):
        self.loss = None
        self.accuracy = None
        
    def __call__(self, x, t):
        self.clear()
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)
        self.accuracy = F.accuracy(y, t)
        chainer.report({'loss': loss, 'accuracy': self.accuracy}, self)
        return loss


In [20]:

gpu_id = 0 #  use gpu
model = Classifier(VGG(class_labels))

if gpu_id >= 0:
    chainer.cuda.get_device(gpu_id).use()  # Make a specified GPU current
    model.to_gpu()  # Copy the model to the GPU

# Setup an optimizer

In [21]:
optimizer = chainer.optimizers.MomentumSGD(0.1)
optimizer.use_cleargrads()
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))

In [22]:
batch = train_iter.next()

In [23]:
xp = model.xp
xp

<module 'cupy' from '/home/komatsu/anaconda2/lib/python2.7/site-packages/cupy/__init__.pyc'>

In [24]:
x = xp.asarray([it[0] for it in batch], dtype=np.float32)
t = xp.asarray([it[1] for it in batch], dtype=np.int32)

In [25]:
x[0].shape

(1, 28, 28)

In [26]:
# predict without Classifier
y = model.predictor(x)
loss = F.softmax_cross_entropy(y, t)
#print(y.data)
print(loss.data)

2.45369696617


In [27]:
y.data

array([[ 0.53663039,  0.02479448, -0.24313273, ...,  0.60040087,
         0.10376104,  0.91054815],
       [ 0.73547   , -0.15444329, -0.33329362, ...,  0.68617147,
        -0.3283636 ,  0.18050759],
       [ 0.69740087,  0.36584252, -0.567967  , ...,  0.57524574,
        -0.48962125,  0.24601667],
       ..., 
       [ 0.18055549,  0.34818384, -0.45530042, ...,  0.50181001,
        -0.35842976, -0.60386521],
       [ 0.15093195, -0.54530537, -0.23916532, ...,  0.47019705,
        -0.97044528, -0.18581873],
       [-0.2049723 , -0.37635037,  0.51578683, ...,  0.24819635,
        -0.41355443,  0.13478515]], dtype=float32)

In [28]:
t[0]

array(7, dtype=int32)

In [ ]:
%%time
from tqdm import tqdm_notebook as tqdm
# from tqdm import tqdm

# run
xp = model.xp

pbar = tqdm(xrange(epochsize))
for epoch in pbar:
    for batch in train_iter:
        # data separation
        x = xp.asarray([it[0] for it in batch], dtype=np.float32)
        t = xp.asarray([it[1] for it in batch], dtype=np.int32)
        # compute grad
        loss = model(x, t)
        model.cleargrads()
        loss.backward()
        optimizer.update()

        # terminate
        if train_iter.is_new_epoch is True:
            break

    # evaluate model   
    sum_loss = 0
    sum_acc = 0
    test_iter_copy = copy.copy(test_iter)
    for test_batch in test_iter_copy:
        # data separation
        x = xp.asarray([it[0] for it in test_batch], dtype=np.float32)
        t = xp.asarray([it[1] for it in test_batch], dtype=np.int32)
        # compute grad
        loss = model(x, t)
        sum_loss += loss.data * len(test_batch)
        sum_acc += model.accuracy.data * len(test_batch)
    mean_loss = sum_loss / testsize
    mean_acc = sum_acc / testsize
#     print(''.format(epoch=epoch))
    print('epoch : {epoch}, Mean loss: {loss}, Mean accuracy: {acc}'.format(epoch=epoch, loss=mean_loss, acc=mean_acc))
    # pbar.set_description('epoch : {epoch}'.format(epoch=epoch))

epoch : 0, Mean loss: 0.0547607652843, Mean accuracy: 0.981999993324
epoch : 1, Mean loss: 0.0490360669792, Mean accuracy: 0.983600020409
epoch : 2, Mean loss: 0.0510660000145, Mean accuracy: 0.984099984169
epoch : 3, Mean loss: 0.042333830148, Mean accuracy: 0.986500024796
epoch : 4, Mean loss: 0.0372350290418, Mean accuracy: 0.988300025463
epoch : 5, Mean loss: 0.0423946641386, Mean accuracy: 0.985899984837
epoch : 6, Mean loss: 0.0409948192537, Mean accuracy: 0.986699998379
epoch : 7, Mean loss: 0.0414143800735, Mean accuracy: 0.986299991608
epoch : 8, Mean loss: 0.0379548147321, Mean accuracy: 0.986199975014
epoch : 9, Mean loss: 0.0414137542248, Mean accuracy: 0.985800027847
epoch : 10, Mean loss: 0.0394147112966, Mean accuracy: 0.987500011921
epoch : 11, Mean loss: 0.030145181343, Mean accuracy: 0.990400016308
epoch : 12, Mean loss: 0.0308239590377, Mean accuracy: 0.990299999714
epoch : 13, Mean loss: 0.037976142019, Mean accuracy: 0.988099992275
epoch : 14, Mean loss: 0.03754238